In [5]:
OPENROUTER_API_KEY = "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"

In [6]:
url = f"https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://github.com/negotiation-research",
    "X-Title": "Negotiation Research"
}

messages = [
    {"role": "user", "content": "Say kiddo kid"}
]

payload = {
    "model": "deepseek/deepseek-r1-0528",
    "messages": messages,
    "temperature": 0.7,
    "max_tokens": 4000,  # Cap at 4000 for safety
}

timeout = 300.0

In [ ]:
import json
import shutil
import time
from pathlib import Path

def send_request(url: str, headers: dict, payload: dict, timeout: float) -> tuple[str | None, str | None]:
    """
    Write request file, poll for response, return (result, error).
    Raises TimeoutError if no response within 10 minutes.
    """
    POLL_DIR = Path("/home/jz4391/openrouter_proxy")
    PROCESSED_DIR = POLL_DIR / "processed"
    POLL_INTERVAL = 0.5
    CLIENT_TIMEOUT = 6000  # 10 minutes

    timestamp = f"{time.time():.6f}".replace('.', '-')
    request_path = POLL_DIR / f"request_{timestamp}.json"
    response_path = POLL_DIR / f"response_{timestamp}.json"

    request_data = {
        "url": url,
        "headers": headers,
        "payload": payload,
        "timeout": timeout,
    }
    with open(request_path, 'w') as f:
        json.dump(request_data, f)
    print(f"[client] Wrote {request_path.name}")

    start = time.time()
    while not response_path.exists():
        if time.time() - start > CLIENT_TIMEOUT:
            # Clean up orphaned request if still there
            if request_path.exists():
                request_path.unlink()
            raise TimeoutError(f"No response after {CLIENT_TIMEOUT}s for {timestamp}")
        time.sleep(POLL_INTERVAL)

    with open(response_path, 'r') as f:
        response_data = json.load(f)

    result, error = response_data["result"], response_data["error"]
    print(f"[client] Got response for {timestamp}: {'error' if error else 'success'}")

    shutil.move(response_path, PROCESSED_DIR / response_path.name)

    return result, error

In [ ]:
send_request(url, headers, payload, timeout)

[client] Wrote request_1768773000-464535.json
[client] Got response for 1768773000-464535: success


('It looks like you\'ve said "Say kiddo kid" — did you mean something like:  \n\n- "Hey kiddo" (a friendly/casual way to address a child or younger person)?  \n- Or "Say \'kiddo\'" (asking someone to repeat the word)?  \n- Or perhaps it\'s a typo/mistake?  \n\nIf you clarify what you meant, I’d be happy to help! 😊',
 None)

: 